In [1]:
import psycopg2
import csv
from tabulate import tabulate
import pandas as pd
import numpy as np
from IPython.display import HTML, display

In [103]:
df = pd.read_csv('athlete_events.csv')
df = df.fillna(0)
df.Age = df.Age.astype('Int32')
df.Height = df.Height.astype('Int32')
df.to_csv('athlete_events_sql.csv')

In [43]:
conn = psycopg2.connect("host=localhost port=5432 dbname=odscourse user=postgres password=secret")
cursor = conn.cursor()

In [4]:
def fetch_all(cursor):
    colnames = [desc[0] for desc in cursor.description]
    records = cursor.fetchall()
    result = (display(HTML(tabulate(
        [{colname:value for colname, value in zip(colnames, record)} for record in records],
        tablefmt='html'))))
    return result

In [106]:
cursor.execute(
    """
    DROP TABLE olympics
    """
)
conn.commit()

In [5]:
query = """
CREATE TABLE IF NOT EXISTS olympics (
    db_id SERIAL PRIMARY KEY,
    id INTEGER,
    name VARCHAR,
    sex VARCHAR,
    age INTEGER,
    height INTEGER,
    weight REAL,
    team VARCHAR,
    NOC VARCHAR,
    games VARCHAR,
    year INTEGER,
    season VARCHAR,
    city VARCHAR,
    sport VARCHAR,
    event VARCHAR,
    medal VARCHAR
)
"""
cursor.execute(query)
conn.commit()

In [6]:
with open('athlete_events_sql.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    for Id, row in enumerate(reader):
        cursor.execute(
            "INSERT INTO olympics VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
            row
        )
conn.commit()

In [6]:
PATH = 'athlete_events_sql.csv'

In [32]:
df = pd.read_csv(PATH)
df.head()

,Unnamed: 0,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,0,1,A Dijiang,M,24,180,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,0
1,1,2,A Lamusi,M,23,170,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,0
2,2,3,Gunnar Nielsen Aaby,M,24,0,0.0,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,0
3,3,4,Edgar Lindenau Aabye,M,34,0,0.0,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,4,5,Christine Jacoba Aaftink,F,21,185,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,0


In [66]:
cursor.execute(
    '''
    SELECT * FROM olympics LIMIT 3    
    '''
)
fetch_all(cursor)

0,1,A Dijiang,M,24,180,80,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,0
1,2,A Lamusi,M,23,170,60,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,0
2,3,Gunnar Nielsen Aaby,M,24,0,0,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,0


# Solutions:

__1. How old were the youngest male and female participants of the 1996 Olympics?__

- 16 and 15
- 14 and 12 +
- 16 and 12
- 13 and 11

In [86]:
cursor.execute(
    '''
SELECT MIN(age) FROM olympics WHERE sex = 'M' AND year = 1996 AND age > 0    
    '''
)
fetch_all(cursor)

cursor.execute(
    '''
SELECT MIN(age) FROM olympics WHERE sex = 'F' AND year = 1996 AND age > 0    
    '''
)
fetch_all(cursor)

14


12


__2. What was the percentage of male gymnasts among all the male participants of the 2000 Olympics? Round the answer to the first decimal.__

*Hint:* here and further if needed drop duplicated sportsmen to count only unique ones. 

- 0.2
- 1.5 +
- 2.5
- 7.7

In [95]:
cursor.execute(
    '''
SELECT ROUND((COUNT(DISTINCT(id)) / (SELECT COUNT(DISTINCT(id))
FROM olympics
    WHERE sex = 'M' AND year = 2000)::numeric) * 100, 1)
FROM olympics
    WHERE sport = 'Gymnastics' AND sex = 'M' AND year = 2000;
    '''
)
fetch_all(cursor)

1.5


__3. What are the mean and standard deviation of height for female basketball players participated in the 2000 Olympics? Round the answer to the first decimal.__

- 178.5 and 7.2
- 179.4 and 10
- 180.7 and 6.7
- 182.4 and 9.1 +

In [123]:
cursor.execute(
    '''
SELECT ROUND(AVG(crit.height)::numeric, 1) AS "mean",
    ROUND(STDDEV(crit.height)::numeric, 1) AS "stddev"
FROM (SELECT DISTINCT id, height FROM olympics
    WHERE sex = 'F' AND sport = 'Basketball' AND year = 2000 AND height > 0) AS crit
    '''
)
fetch_all(cursor)

182.4,9.1


__4. Find a sportsperson participated in the 2002 Olympics, with the highest weight among other participants of the same Olympics. What sport did he or she do?__

- Judo
- Bobsleigh +
- Weightlifting
- Boxing

In [128]:
cursor.execute(
    '''
SELECT sport
FROM olympics
    WHERE year = 2002 AND weight=(SELECT MAX(weight) FROM olympics WHERE year = 2002)
    '''
)
fetch_all(cursor)

Bobsleigh


__5. How many times did Pawe Abratkiewicz participate in the Olympics held in different years?__

- 0
- 1 
- 2
- 3 +

In [129]:
cursor.execute(
    '''
SELECT COUNT(DISTINCT(year))
FROM olympics
    WHERE name = 'Pawe Abratkiewicz'
    '''
)
fetch_all(cursor)

3


__6. How many silver medals in tennis did Australia win at the 2000 Olympics?__

- 0
- 1 
- 2 +
- 3 

In [142]:
cursor.execute(
    '''
SELECT COUNT(*)
FROM olympics
    WHERE medal = 'Silver' AND sport = 'Tennis' AND team = 'Australia' AND year = 2000
    '''
)
fetch_all(cursor)

2


__7. Is it true that Switzerland won fewer medals than Serbia at the 2016 Olympics? Do not consider NaN values in _Medal_ column.__ 

- Yes +
- No

In [14]:
cursor.execute(
    '''
SELECT(SELECT COUNT(*)
FROM olympics
    WHERE team = 'Switzerland' AND year = 2016 AND medal != '0') <
(SELECT COUNT(*)
FROM olympics
    WHERE team = 'Serbia' AND year = 2016 AND medal != '0') AS ans
    '''
)
fetch_all(cursor)

True


__8. What age category did the fewest and the most participants of the 2014 Olympics belong to?__
- [45-55] and [25-35) correspondingly +
- [45-55] and [15-25) correspondingly
- [35-45] and [25-35) correspondingly
- [45-55] and [35-45) correspondingly

In [28]:
cursor.execute(
    '''
    DROP TABLE age_intervals
    '''
)
conn.commit()

In [29]:
cursor.execute(
    '''
CREATE TABLE age_intervals(interval VARCHAR, fewest INTEGER, most INTEGER);
INSERT INTO age_intervals VALUES('[45-55]/[23-35)', (SELECT COUNT(*) FROM olympics WHERE age >= 45 AND age <= 55 AND year = 2014),
                                    (SELECT COUNT(*) FROM olympics WHERE age >= 25 AND age < 35 AND year = 2014));
INSERT INTO age_intervals VALUES('[45-55]/[15-25)', (SELECT COUNT(*) FROM olympics WHERE age >= 45 AND age <= 55 AND year = 2014),
                                    (SELECT COUNT(*) FROM olympics WHERE age >= 15 AND age < 25 AND year = 2014));
INSERT INTO age_intervals VALUES('[35-45]/[25-35)', (SELECT COUNT(*) FROM olympics WHERE age >= 35 AND age <= 45 AND year = 2014),
                                    (SELECT COUNT(*) FROM olympics WHERE age >= 25 AND age < 35 AND year = 2014));
INSERT INTO age_intervals VALUES('[45-55]/[35-45)', (SELECT COUNT(*) FROM olympics WHERE age >= 45 AND age < 55 AND year = 2014),
                                    (SELECT COUNT(*) FROM olympics WHERE age >= 35 AND age < 45 AND year = 2014));
    '''
)
conn.commit()

cursor.execute(
    '''
SELECT * FROM age_intervals
    '''
)
fetch_all(cursor)

[45-55]/[23-35),5,2594
[45-55]/[15-25),5,2027
[35-45]/[25-35),266,2594
[45-55]/[35-45),4,265


__9. Is it true that there were Summer Olympics held in Lake Placid? Is it true that there were Winter Olympics held in Sankt Moritz?__

- Yes, Yes
- Yes, No
- No, Yes +
- No, No 

In [34]:
cursor.execute(
    '''
SELECT (SELECT COUNT(*) > 0
FROM olympics
    WHERE city = 'Lake Placid' AND season = 'Summer') AS placid,
(SELECT COUNT(*) > 0
FROM olympics
    WHERE city = 'Sankt Moritz' AND season = 'Winter') AS moritz
    '''
)
fetch_all(cursor)

False,True


__10. What is the absolute difference between the number of unique sports at the 1995 Olympics and 2016 Olympics?__

- 16
- 24
- 26
- 34 +

In [44]:
cursor.execute(
    '''
SELECT ABS((SELECT COUNT (DISTINCT sport)
FROM olympics
    WHERE year = 1995) -
    (SELECT (SELECT COUNT (DISTINCT sport)
FROM olympics
    WHERE year = 2016)))
    '''
)
fetch_all(cursor)

34
